## General dataframe

In [1]:
import pandas as pd
fh = '../files/tickets-gen-all.csv'
df = pd.read_csv(fh, index_col=0, parse_dates=['created', 'opened_at', 'updated_on', 'resolved'])

In [15]:
df.columns

Index(['created', 'caller', 'opened_at', 'opened_by', 'priority', 'state',
       'category', 'subcategory', 'assignment_group', 'bpl_location',
       'nyp_locaiton', 'resolved', 'assigned_to', 'system', 'closed_at',
       'closed_by', 'mat_source', 'reassignment_count', 'reopen_count',
       'resolved_by', 'updated_on', 'updated_by'],
      dtype='object')

In [3]:
df['assignment_group'].unique()

array(['BKOPS CAT', 'BKOPS ACQ', 'BKOPS CPRC', 'BKOPS LOG', 'BKOPS RECAP',
       'BKOPS SEL', 'IT ILS', 'IT Branch Library Support', nan,
       'BKOPS ITIL', 'BKOPS LOG - MyLibNYC'], dtype=object)

In [4]:
df['category'].unique()

array(['Cataloging', 'Acquisitions', 'Selection', 'Logistics',
       'Offsite - ReCAP', 'Collections Processing', nan], dtype=object)

In [5]:
df['state'].unique()

array(['Closed', 'Active', 'Awaiting User Info', 'Awaiting Vendor', 'New',
       'Resolved'], dtype=object)

In [6]:
df['system'].unique()

array(['NYPL Circulating', 'NYPL Research', 'BPL Circulating', nan],
      dtype=object)

## Tickets by state in each dept

In [13]:
frames = []
for library, ldf in df.groupby('system'):
    for dept, dldf in ldf.groupby('category'):
        for state, sdldf in dldf.groupby('state'):
            frames.append(
                {'library': library, 'dept': dept, 'state': state, 'tickets': sdldf.shape[0]}
            )
df_by_state = pd.DataFrame(frames)
df_by_state.to_csv('../data-display/states-by-dept.csv', index=False)

## Active users

### user by dept lists

In [28]:
cat2group = {
    'Cataloging': ['BKOPS CAT'],
    'Acquisitions': ['BKOPS ACQ'],
    'Selection': ['BKOPS SEL'],
    'Logistics': ['BKOPS LOG', 'BKOPS LOG - MyLibNYC'],
    'Offsite - ReCAP': ['BKOPS RECAP'],
    'Collections Processing': ['BKOPS CPRC']
}
group2cat = {
    'BKOPS CAT': 'Cataloging',
    'BKOPS ACQ': 'Acquisitions',
    'BKOPS SEL': 'Selection',
    'BKOPS LOG': 'Logistics',
    'BKOPS LOG - MyLibNYC': 'Logistics',
    'BKOPS RECAP': 'Offsite - ReCAP',
    'BKOPS CPRC': 'Collections Processing',
}

In [34]:
frames = []
for dept, values in cat2group.items():
#     cdf = df[(df['category'] == dept) & (df['assignment_group'].isin(values))]
#     print(f'correct: {dept}={cdf.shape[0]}')
    idf = df[(df['category'] == dept) & (~df['assignment_group'].isin(values))]
    for dst, didf in idf.groupby('assignment_group'):
        if dst in group2cat.keys():
            frames.append({'dept_from': dept, 'dept_to': group2cat[dst], 'tickets': didf.shape[0] })
misassigned_connections = pd.DataFrame(frames)
misassigned_connections.head()

,dept_from,dept_to,tickets
0,Cataloging,Acquisitions,158
1,Cataloging,Collections Processing,319
2,Cataloging,Logistics,1
3,Cataloging,Offsite - ReCAP,17
4,Cataloging,Selection,83


In [35]:
misassigned_connections.to_csv('../data-display/misassigned-connections.csv', index=False)